In [25]:
import numpy as np
from numpy import cos, sin, pi
import matplotlib.pyplot as plt
from libs.XY_model import XYSystem
import cv2
from tqdm import tqdm

In [26]:
def get_curl(x):    
    N = x.shape[0]
    curl = []
    ux = cos(x)
    uy = sin(x)
    for i in range(N):
        cu = []
        for j in range(N):
            c = (ux[i, j] - ux[(i+1)%N, (j+1)%N]) + (uy[(i+1)%N, j] - uy[i, (j+1)%N]) 
            c += np.sum(ux[(i-1)%N, (j-1)%N : (j+3)%N]) - \
                 np.sum(ux[(i+2)%N, (j-1)%N : (j+3)%N]) + \
                 \
                 np.sum(uy[(i-1)%N : (i+3)%N, (j-1)%N]) - \
                 np.sum(uy[(i-1)%N : (i+3)%N, (j+2)%N])
            
            c += np.sum(ux[(i-2)%N, (j-2)%N : (j+4)%N]) - \
                 np.sum(ux[(i+3)%N, (j-2)%N : (j+4)%N]) + \
                 \
                 np.sum(uy[(i-2)%N : (i+4)%N, (j-2)%N]) - \
                 np.sum(uy[(i-2)%N : (i+4)%N, (j+3)%N])
                 
          #   c += np.sum(ux[(i-3)%N, (j-3)%N : (j+5)%N]) - \
          #        np.sum(ux[(i+4)%N, (j-3)%N : (j+5)%N]) + \
          #        \
          #        np.sum(uy[(i-3)%N : (i+5)%N, (j-3)%N]) - \
          #        np.sum(uy[(i-3)%N : (i+5)%N, (j+4)%N]) / 49
            cu.append(c)
        curl.append(cu)
             
    curl = np.array(curl)
    curl /= np.sum(curl)
    return curl

In [27]:
def list2matrix(S):
        N=int(np.size(S))
        L = int(np.sqrt(N))
        S=np.reshape(S,(L,L))
        return S

In [28]:
def plot_heatmap(spin_config, curl, temp, name = 'result', heat = True, just_heat = False, spin = False):
    N = spin_config.shape[0]
    X, Y = np.meshgrid(np.arange(0,N),np.arange(0, N))
    U = np.cos(spin_config)
    V = np.sin(spin_config)
    plt.figure(figsize=(4,4),dpi=600, facecolor='white')
    if(heat):
        plt.imshow(curl)
        plt.colorbar()
    if(just_heat):
        plt.imshow(curl)
        plt.colorbar()
        plt.savefig(f'./images/{name}.png', dpi = 600)
        plt.close()
        return
    if(spin):
        plt.imshow(U)
        plt.colorbar()
    Q = plt.quiver(X, Y, U, V, units='width')
    qk = plt.quiverkey(Q, 0.1, 0.1, 1, r'$spin$', labelpos='E',
                coordinates='figure')
    plt.title('T=%.2f'%temp+', #spins='+str(N)+'x'+str(N))
    plt.savefig(f'./images/{name}.png', dpi = 600)
    plt.close()

In [29]:
temp = [0.01]
N = [128]

In [30]:
for n in tqdm(range(len(N))):    
    for i in range(len(temp)):
        xy1 = XYSystem(temperature = temp[i], width = N[n])
        xy1.equilibrate(show=False)
        x = list2matrix(xy1.spin_config)
        cu = get_curl(x)
        plot_heatmap(x, cu, temp[i], f'mix3_{temp[i]}_{N[n]}')
        # plot_heatmap(x, cu, temp[i], f'vec_{temp[i]}_{N[n]}', False)
        # plot_heatmap(x, cu, temp[i], f'curl_{temp[i]}_{N[n]}', False, True)
        # plot_heatmap(x, cu, temp[i], f'cos_{temp[i]}_{N[n]}', False, False, True)

  0%|          | 0/1 [00:00<?, ?it/s]


equilibrium state is reached at T=0.0
#sweep=501
energy=-1.90


  0%|          | 0/1 [05:17<?, ?it/s]


equilibrium state is reached at T=0.1
#sweep=501
energy=-1.90


KeyboardInterrupt: 